In [1]:
from numpy import array
import mtt
wiring = mtt.Wiring.bare(['Protein 1.1', 'Protein 1.2', 'Protein 1.3',
                        'Protein 2.1', 'Protein 2.2', 'Protein 2.3',
                        'Protein 3.1', 'Protein 3.2', 'Protein 3.3'])

# Configure Connections

In [2]:
# Protein 1
wiring.blocks[0].Afree.connect(wiring.blocks[1].Atot)
wiring.blocks[0].Ctot.connect( wiring.blocks[0].Cfree)
wiring.blocks[1].Ctot.connect( wiring.blocks[2].Atot)
wiring.blocks[1].Ctot.connect( wiring.blocks[1].Cfree)
wiring.blocks[2].Ctot.connect( wiring.blocks[3].Btot)
wiring.blocks[2].Ctot.connect( wiring.blocks[2].Cfree)
# Protein 2
wiring.blocks[3].Afree.connect(wiring.blocks[4].Atot)
wiring.blocks[3].Ctot.connect( wiring.blocks[3].Cfree)
wiring.blocks[4].Ctot.connect( wiring.blocks[5].Atot)
wiring.blocks[4].Ctot.connect( wiring.blocks[4].Cfree)
wiring.blocks[5].Ctot.connect( wiring.blocks[6].Btot)
wiring.blocks[5].Ctot.connect( wiring.blocks[5].Cfree)
# Protein 3
wiring.blocks[6].Afree.connect(wiring.blocks[7].Atot)
wiring.blocks[6].Ctot.connect( wiring.blocks[6].Cfree)
wiring.blocks[7].Ctot.connect( wiring.blocks[8].Atot)
wiring.blocks[7].Ctot.connect( wiring.blocks[7].Cfree)
wiring.blocks[8].Ctot.connect( wiring.blocks[0].Btot)
wiring.blocks[8].Ctot.connect( wiring.blocks[8].Cfree)

# Configure Currents

In [3]:
for k in range(0,9,3):
    wiring.blocks[k+0].Atot = 865. #86.5 #865.
    wiring.blocks[k+0].Btot = 0.
    wiring.blocks[k+0].KDfw = 4. #10. # 4.
    wiring.blocks[k+0].kr = 1000. #100. #1000.
    wiring.blocks[k+0].kdeg = 1.
    wiring.blocks[k+0].Dfree = 10.
    wiring.blocks[k+0].KDrv = 10.
    wiring.blocks[k+0].ratC = 0.
#     print('ratC = {}'.format(wiring.blocks[k+0].ratC))
    wiring.blocks[k+0].n = 2
    wiring.blocks[k+0].A_FB_EN = False
    wiring.blocks[k+0].Afree.setObservable('Afree')

    wiring.blocks[k+1].Atot = 0.865
    wiring.blocks[k+1].Btot = 10.
    wiring.blocks[k+1].KDfw = 10.
    wiring.blocks[k+1].kr = 100.
    wiring.blocks[k+1].kdeg = 1.
    wiring.blocks[k+1].Dfree = 10.
    wiring.blocks[k+1].KDrv = 10.
    wiring.blocks[k+1].ratC = 0.
    wiring.blocks[k+1].Ctot.setObservable('Ctot 2')

    wiring.blocks[k+2].Btot = 10.
    wiring.blocks[k+2].KDfw = 10.
    wiring.blocks[k+2].kr = 20.
    wiring.blocks[k+2].kdeg = 1.
    wiring.blocks[k+2].Dfree = 10.
    wiring.blocks[k+2].KDrv = 10.
    wiring.blocks[k+2].ratC = 0.
    wiring.blocks[k+2].Ctot.setObservable('Ctot 3')

In [4]:
from mtt import addSRAMRule, binrep
# add observables manually
m = mtt.MTT(wiring=wiring)
program = m.compileSRAM()
rules = program.getRoutingRulesBinaryMatrix()

# Sung's original connections
# connect ICtot_copyP3 to SRAM wire 88
rules = addSRAMRule(rules,0,2,11,88)
# connect ICtot_copyP3 to SRAM wire 89
rules = addSRAMRule(rules,1,2,11,89)
# connect ICtot_copyP3 to SRAM wire 90
rules = addSRAMRule(rules,2,2,11,90)
# connect IAfree_copy2 to SRAM wire 91
rules = addSRAMRule(rules,0,1,11,91)
# connect ICtot_copyP3 to SRAM wire 92
rules = addSRAMRule(rules,1,1,11,92)
# connect ICtot_copyP3 to SRAM wire 93
rules = addSRAMRule(rules,2,1,11,93)

# five extra ADC connections from group 2 block 1 - what are they for?
rules = addSRAMRule(rules,2,1,18,94)
rules = addSRAMRule(rules,2,1,19,95)
rules = addSRAMRule(rules,2,1,12,96)
rules = addSRAMRule(rules,2,1,7,97)
rules = addSRAMRule(rules,2,1,25,98)

# use ADC 0~5
# for i in range(88,93+1):
#     rules = addSRAMRule(rules,0,0,31,i)
# print(binrep(rules))
# print(rules)

In [5]:
# extra ADC connections?
for i in range(76,99+1):
    rules = addSRAMRule(rules,0,0,31,i)

In [6]:
# hardware simulation
m.reset()
r = m.simulate(0,1,100,block_output=True)
# for k in range(3):
#     print('block {}: Afree = {}'.format(k,model.blocks[k].Afree))
#     print('block {}: Bfree = {}'.format(k,model.blocks[k].Bfree))
#     print('block {}: rate_fw = {}'.format(k,model.blocks[k].rate_fw))
#     print('block {}: rate_rv = {}'.format(k,model.blocks[k].rate_rv))
#     print('block {}: rate = {}'.format(k,model.blocks[k].rate))
# print(r)
m.plot()

In [7]:
from scipy.io import savemat
from os.path import expanduser
sr = m.compileShiftReg()
prog_vect = sr.getProgrammingVector()
savemat(expanduser('~/devel/src/mtt/case-studies/jon-repressilator-02282019/sr_only/sr.mat'), {'prog_vect': prog_vect})

In [8]:
sram_matrix = program.convertToSRAMMatrix(rules)
print(sram_matrix)
savemat(expanduser('~/devel/src/mtt/case-studies/jon-repressilator-02282019/sram_only/sram.mat'), {'in_all': sram_matrix})

In [9]:
from scipy.io import loadmat
from os.path import expanduser
m_matlab = loadmat(expanduser('~/Downloads/repressilator_from_matlab.mat'))['in_all']
sram_matrix - m_matlab

In [10]:
# render rules
#from numpy import array
#bin_rules = array([bin(rule)[2:].zfill(10) for rule in rules])
#print(bin_rules.reshape((-1,1)))

In [11]:
model = mtt.MTT(wiring)
model.draw(3)

# Generate Program

In [12]:
sram = model.compileSRAM()
import pprint
pp = pprint.PrettyPrinter(indent=2, depth=1, width=10)
pp.pprint(sram.dump())